<a href="https://colab.research.google.com/github/mariddc/Projeto-RL-Highway-gym/blob/main/new_HIGHWAY_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GYM - STABLE BASELINES
##  🏎 Ambiente Highway 🏎

O primeiro passo é instalar as bibliotecas necessárias para o notebook.

In [1]:
# Install environment and agent
!pip install highway-env
# TODO: we use the bleeding edge version because the current stable version does not support the latest gym>=0.21 versions. Revert back to stable at the next SB3 release.
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 7.5 MB/s 
     |████████████████████████████████| 21.8 MB 1.5 MB/s 
     |████████████████████████████████| 721 kB 48.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827650 sha256=e3069acd363300b1add0dd6a7e56efe0f39afe6e3ef662729351cdd8dcfa31eb
  Stored in directory: /root/.cache/pip/wheels/4d/c8/dc/d08577bffa680f083f04448e81e4f176d0e45a13eb16f93c0c
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/carlosluis/stable-baselines3 (to revision fix_tests) to

In [2]:
# Instalação de bibliotecas necessárias para visualizar o ambiente no Colab
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils xvfb
0 upgraded, 3 newly installed, 0 to remove and 5 not upgraded.
Need to get 994 kB of archives.
After this operation, 2,982 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.11 [785 kB]
Fetched 994 kB in 1s (762 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 123991 files 

In [3]:
!pip install moviepy
!pip install --upgrade imageio-ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 53.6 MB/s 


Agora vamos importar o ambiente do _gym_ e diferentes tipos de algoritmos que queremos treinar da biblioteca _stable baselines_. Além disso, também importamos ferramentas para, mais tarde, avaliarmos e visualizarmos o desempenho do agente.

In [4]:
# Environment
import gym
import highway_env

# Agent
from stable_baselines3 import PPO, DQN

# Evaluation
from stable_baselines3.common.evaluation import evaluate_policy 

# Visualization
from gym.wrappers import RecordVideo

Configurando o display para permitir a visualização dos resultados.

In [5]:
import pyvirtualdisplay

# Cria um display dentro do Colab
_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

In [6]:
from highway_env.envs.common.graphics import EnvViewer

class RenderWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)

    def render(self, mode="rgb_array"):
        """
        Render the environment.
        Create a viewer if none exists, and use it to render an image.
        :param mode: the rendering mode
        """
        self.rendering_mode = mode

        if self.viewer is None:
            self.viewer = EnvViewer(self)

        self.enable_auto_render = True

        self.viewer.display()

        if not self.viewer.offscreen:
            self.viewer.handle_events()
        if mode == 'rgb_array':
            image = self.viewer.get_image()
            return image

Criamos o ambiente que queremos consultando as possibilidades da biblioteca [_gymnasium_](https://gymnasium.farama.org/environments/third_party_environments/). Depois, escolhemos o algoritmo do [_stable baselines_](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html) que será testado, nesse caso o [PPO](https://openai.com/blog/openai-baselines-ppo/) (Proximal Policy Optimization) e rodamos um episódio inicial.

In [7]:
env = gym.make("racetrack-v0", render_mode="rgb_array")
env = RenderWrapper(env)
env = RecordVideo(env, video_folder="videos", episode_trigger=lambda e: True)

env.metadata['render_fps'] = 10
env.unwrapped.render_mode = "rgb_array"
env.unwrapped.set_record_video_wrapper(env)

env = env 

obs, done = env.reset(seed=45), False

for i in range(150):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)

    #done = terminated or truncated

env.close()

/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:21: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  f"It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: {observation_space.dtype}. "
/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:26: UserWarning: WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  "It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. "


[MoviePy] >>>> Building video /content/videos/rl-video-episode-0.mp4
[MoviePy] Writing video /content/videos/rl-video-episode-0.mp4


100%|██████████| 151/151 [00:01<00:00, 79.46it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/videos/rl-video-episode-0.mp4 



In [8]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('videos/rl-video-episode-0.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


In [9]:
# new env for training
train_env = gym.make("racetrack-v0", render_mode="rgb_array")

obs, done = train_env.reset(seed=45), False

# Setting model
model = PPO('MlpPolicy', train_env, seed=8, verbose=1)

# Training model
model.learn(total_timesteps=30000)

/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:21: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  f"It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: {observation_space.dtype}. "
/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:26: UserWarning: WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  "It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. "


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.5e+03  |
|    ep_rew_mean     | 18.3     |
| time/              |          |
|    fps             | 24       |
|    iterations      | 1        |
|    time_elapsed    | 84       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.5e+03     |
|    ep_rew_mean          | 19.9        |
| time/                   |             |
|    fps                  | 24          |
|    iterations           | 2           |
|    time_elapsed         | 168         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.003215849 |
|    clip_fraction        | 0.0126      |
|    clip_range           | 0.2         |
|    entropy_loss   

In [10]:
# evaluation
eval_env = gym.make("racetrack-v0")
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

print(f"Recompensa Média: {mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:71: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Recompensa Média: 333.31 +/- 444.5714837931524


In [11]:
train_env = RenderWrapper(train_env)
train_env = RecordVideo(train_env, video_folder="videos", episode_trigger=lambda e: True)

train_env.metadata['render_fps'] = 10
train_env.unwrapped.render_mode = "rgb_array"
train_env.unwrapped.set_record_video_wrapper(train_env)

train_env = train_env 

/usr/local/lib/python3.7/dist-packages/gym/wrappers/record_video.py:76: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  f"Overwriting existing videos at {self.video_folder} folder "


In [12]:
mp4 = open('videos/rl-video-episode-0.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)